### Build the dataframe from the raw files

In [1]:
# Import required packages
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [2]:
def parse_xml(xml_file):

    # Load the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Define the namespace
    namespace = {
        'atom': 'http://www.w3.org/2005/Atom',
        'nitf': 'http://iptc.org/std/NITF/2006-10-18/',
        'dc': 'http://purl.org/dc/elements/1.1/'
    }

    # Extract the title
    title = root.find('.//atom:title', namespace).text

    # Extract the published date
    published_date = root.find('.//atom:published', namespace).text
    published_date = published_date.split('T')[0]

    # Extract the publisher
    publisher = root.find('.//metadata/publicationInfo/publicationName').text


    # Extract the body text
    body_text = root.find('.//nitf:body.content//bodyText//p', namespace)

    # Extract all body paragraphs and join them into a single line
    body_paragraphs = []
    body_text_element = root.find('.//nitf:body.content//bodyText', namespace)
    if body_text_element is not None:
        for p_element in body_text_element.findall('.//p', namespace):
            if p_element.text:
                body_paragraphs.append(p_element.text.strip())
    text = ' '.join(body_paragraphs)

    # Check for empty text bodies
    if text == '':
        print('No text body in file ' + file_path)
        return

    # Create a dictionary with the extracted values
    data_dict = {
        'title': [title],
        'publication_date': [published_date],
        'publisher': [publisher],
        'text': [text]
    }

    # Create a pandas DataFrame
    df = pd.DataFrame(data_dict)

    # Return the resulting DataFrame
    return df

In [3]:
# Root Directory of the data
folder_path = 'c:\\Friendcloud\\_University\\_Masterarbeit\\Data\\Nexis\\'

# Go through all subdirectories and fill raw_data with the data
raw_data = []
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xml'):
            try:
                file_path = os.path.join(root, file)
                data = parse_xml(file_path)
                raw_data.append(data)
            except:
                print('Error in file ' + file_path)

# Build pandas DataFrame
df_raw = pd.concat(raw_data, ignore_index=True)

No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\52W90F11JB1632YH0000000_Sieben_Milliarden.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\531SHG91DXG7T2TT0000000_Spielen.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\55118YJ1DXG7T2020000000_Laborfleisch_Hamburger_aus_Stammzell.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\55S47DN1DXG7T36M0000000_Nano_in_der_Küche.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\592FP641JB1634H70000000_Sieht_schon_wie_Fleisch_aus.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\5933KRH1DXG7T0JY0000000_Die_Geschichte_hinter_dem_Hamburger.xml
No text body in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\AppenzellerZeitung\59636D21JB16304R0000000_Bürolisten_mit_Metz

In [4]:
df_raw

,title,publication_date,publisher,text
0,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,Der AP-Überblick am Nachmittag,2018-06-21,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...
6774,Kein Titel,2023-02-21,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
6775,Leben und schmecken lassen,2023-02-21,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
6776,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
6777,Essen aus dem Labor,2009-12-08,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [5]:
# Add year of publication to raw dataset
df_raw['publication_date'] = pd.to_datetime(df_raw['publication_date'], errors='coerce')
df_raw['publication_year'] = df_raw['publication_date'].dt.year
df_raw = df_raw[['title', 'publication_date', 'publication_year', 'publisher', 'text']]

In [6]:
df_raw

,title,publication_date,publication_year,publisher,text
0,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...
6774,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
6775,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
6776,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
6777,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [12]:
df_raw.to_pickle("../Data/df_raw.pkl")

### Data preprocessing

In [13]:
df_processed = pd.read_pickle("../Data/df_raw.pkl")

In [15]:
df_processed

,title,publication_date,publication_year,publisher,text
0,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...
6774,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
6775,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
6776,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
6777,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [16]:
df_processed.to_pickle("../Data/df.pkl")